In [1]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import community.community_louvain as community_louvain
import funkcie as fn
fn.setup_directories()
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import collections

#### NAČÍTANIE DÁT

In [2]:
G = fn.load_data("facebook_combined.txt")
pos = nx.spring_layout(G, seed=42)
fn.draw_graph_raw(G, pos)

Načítavam dáta z: facebook_combined.txt
-> Uzlov: 4039
-> Hrán: 88234


#### EXPERIMENT A: Vlpyv hyperparametra resolution na výsledky modelu Louvain

In [3]:
# 1. Definícia rozsahu resolution
resolutions = np.arange(0, 2.1, 0.1)

results_exp_a = []
best_modularity = -1
best_resolution = 1.0
best_partition = None

# --- VÝPOČTY ---
for r in resolutions:
    r = round(r, 1)
    
    try:
        partition = community_louvain.best_partition(G, resolution=r, random_state=42)
        mod = community_louvain.modularity(partition, G)
        n_comms = len(set(partition.values()))
        
        results_exp_a.append({
            "Resolution": r,
            "Modularita": mod,
            "Počet komunít": n_comms
        })
        
        if mod > best_modularity:
            best_modularity = mod
            best_resolution = r
            best_partition = partition
            
    except Exception as e:
        pass

df_a = pd.DataFrame(results_exp_a)

# -------------------------------------------------------
# 2. Uloženie GRAFU
# -------------------------------------------------------
fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'tab:blue'
ax1.set_xlabel('Resolution Parameter')
ax1.set_ylabel('Modularita', color=color, fontweight='bold')
ax1.plot(df_a['Resolution'], df_a['Modularita'], color=color, marker='o', markersize=4, linewidth=2)
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True, alpha=0.3)

ax2 = ax1.twinx()  
color = 'tab:red'
ax2.set_ylabel('Počet komunít', color=color, fontweight='bold')
ax2.plot(df_a['Resolution'], df_a['Počet komunít'], color=color, marker='s', markersize=4, linestyle='--', linewidth=2)
ax2.tick_params(axis='y', labelcolor=color)

ax1.set_xticks(np.arange(0, 2.1, 0.1))
plt.title(f"Experiment A: Vplyv Resolution ({resolutions[0]} - {resolutions[-1]})")
plt.savefig("img/experiment_A_resolution_fine.png", dpi=300, bbox_inches='tight')
plt.close()

# -------------------------------------------------------
# 3. Uloženie TABUĽKY výsledkov do PNG
# -------------------------------------------------------
# Príprava formátovaných dát pre tabuľku (textové reťazce)
# Týmto vynútime, že int bude int a float bude float s presným počtom desatinných miest
cell_text = []
for index, row in df_a.iterrows():
    cell_text.append([
        f"{row['Resolution']:.1f}",
        f"{row['Modularita']:.4f}",
        f"{int(row['Počet komunít'])}"
    ])

# Nastavenie výšky obrázka
num_rows = len(df_a)
# Pridáme trochu viac miesta navrch kvôli nadpisu
fig_height = (num_rows * 0.3) + 1.0 

fig_tbl, ax_tbl = plt.subplots(figsize=(6, fig_height))
ax_tbl.axis('tight')
ax_tbl.axis('off')

# Vytvorenie tabuľky
table = ax_tbl.table(cellText=cell_text,
                     colLabels=["Resolution", "Modularita", "Počet komunít"],
                     cellLoc='center',
                     loc='center')

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

# Nastavenie nadpisu
plt.title("Detailné výsledky Experimentu A", y=1.0, pad=20, fontsize=12, fontweight='bold')

plt.savefig("img/experiment_A_tabulka.png", dpi=300, bbox_inches='tight')
plt.close()

# -------------------------------------------------------
# 4. Uloženie vizualizácie
# -------------------------------------------------------
fn.draw_communities(G, best_partition, pos, 
                   f"Najlepší model (Louvain res={best_resolution})", 
                   "img/best_louvain_model.png")

#### EXPERIMENT B: Analýza veľkosti komunít

In [4]:
# 1. Načítanie modelu z Experimentu A
# Použijeme premennú 'best_resolution', ktorú vypočítal predchádzajúci kód.
# Ak by náhodou neexistovala, nastavíme 1.0 ako fallback.
try:
    final_res = best_resolution
    print(f"Používam optimalizované resolution z Exp A: {final_res}")
except NameError:
    final_res = 1.0
    print("Premenná z Exp A chýba, používam default resolution=1.0")

# Výpočet rozdelenia (Louvain)
final_partition = community_louvain.best_partition(G, resolution=final_res, random_state=42)

# Získanie veľkostí a zoradenie od najväčšej po najmenšiu
community_counts = collections.Counter(final_partition.values())
sorted_sizes = sorted(community_counts.values(), reverse=True)

# -------------------------------------------------------
# 2. Uloženie GRAFU
# -------------------------------------------------------
plt.figure(figsize=(10, 6))

ranks = range(1, len(sorted_sizes) + 1)
plt.bar(ranks, sorted_sizes, color='tab:green', edgecolor='black', alpha=0.75)

plt.title(f"Veľkosť komunít zoradená podľa poradia (Louvain, res={final_res})")
plt.xlabel("Poradie komunity (Rank)")
plt.ylabel("Počet členov ($N_c$)")

# Úprava osi X pre lepšiu čitateľnosť
if len(ranks) <= 20:
    plt.xticks(ranks)
else:
    plt.locator_params(axis='x', integer=True)

plt.grid(axis='y', linestyle='--', alpha=0.5)

# Pridanie hodnoty nad prvý stĺpec
plt.text(ranks[0], sorted_sizes[0], f" {sorted_sizes[0]}", ha='center', va='bottom', fontweight='bold')

plt.savefig("img/experiment_B_sizes_sorted.png", dpi=300, bbox_inches='tight')
plt.close()

# -------------------------------------------------------
# 3. Uloženie TABUĽKY "Top 5 Komunít"
# -------------------------------------------------------
# Príprava dát: Zoradíme komunity aj s ich ID
sorted_comms_with_ids = sorted(community_counts.items(), key=lambda x: x[1], reverse=True)

top_5_data = []
total_nodes = G.number_of_nodes()

for i, (comm_id, size) in enumerate(sorted_comms_with_ids[:5]):
    share = (size / total_nodes) * 100
    top_5_data.append([
        str(i + 1),               # Rank
        str(comm_id),             # ID
        str(size),                # Size
        f"{share:.1f} %"          # Share
    ])

# Vytvorenie tabuľky
fig_tbl, ax_tbl = plt.subplots(figsize=(6, 2.5))
ax_tbl.axis('tight')
ax_tbl.axis('off')

columns = ["Rank", "Cluster ID", "Veľkosť ($N_c$)", "Podiel na sieti"]

table = ax_tbl.table(cellText=top_5_data,
                     colLabels=columns,
                     cellLoc='center',
                     loc='center')

table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 1.5)

plt.title("Top 5 najväčších komunít", y=1.0, pad=20, fontsize=12, fontweight='bold')
plt.savefig("img/experiment_B_top5.png", dpi=300, bbox_inches='tight')
plt.close()

Používam optimalizované resolution z Exp A: 1.0


#### EXPERIMENT C: Identifikácia centrálnych uzlov

In [5]:
# 1. Využitie premenných z Exp B
# Ak by premenné neexistovali, nastavíme default
try:
    final_res = best_resolution
except NameError:
    final_res = 1.0

# Prepočet modelu
final_partition = community_louvain.best_partition(G, resolution=final_res, random_state=42)

# Zoradenie komunít podľa veľkosti
community_counts = collections.Counter(final_partition.values())
sorted_comms = sorted(community_counts.items(), key=lambda x: x[1], reverse=True)

# 2. Analýza Hubov pre Top 5 komunít
hub_data = []

for i, (comm_id, size) in enumerate(sorted_comms[:5]):
    # A. Vytvoríme pod-graf len pre túto komunitu
    nodes_in_comm = [n for n, c in final_partition.items() if c == comm_id]
    subgraph = G.subgraph(nodes_in_comm)
    
    # B. Nájdeme uzol s najvyšším stupňom vnútri komunity
    degrees = dict(subgraph.degree())
    top_node = max(degrees, key=degrees.get)
    internal_degree = degrees[top_node]
    
    # C. Výpočet pokrytia (%)
    coverage = (internal_degree / (size - 1)) * 100 if size > 1 else 100
    
    hub_data.append([
        str(i + 1),                   # Rank
        str(comm_id),                 # ID
        str(size),                    # Veľkosť
        str(top_node),                # Hub ID
        str(internal_degree),         # Stupeň
        f"{coverage:.1f} %"           # Pokrytie
    ])

# 3. Vytvorenie tabuľky
# Skrátené názvy stĺpcov aby sa zmestili
columns = [
    "Rank", 
    "Komunita ID", 
    "Veľkosť", 
    "Hub ID", 
    "Max. Stupeň", 
    "Pokrytie"
]

df_hub = pd.DataFrame(hub_data, columns=columns)

# Nastavenie výšky obrázka
num_rows = len(df_hub)
fig_height = (num_rows * 0.3) + 1.0 

fig_tbl, ax_tbl = plt.subplots(figsize=(8, fig_height)) # Mierna úprava šírky
ax_tbl.axis('tight')
ax_tbl.axis('off')

# Vykreslenie tabuľky
table = ax_tbl.table(cellText=df_hub.values,
                     colLabels=columns,
                     cellLoc='center',
                     loc='center')

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

# Nadpis
plt.title(f"Analýza centrálnych uzlov (Louvain res={final_res})", 
          y=1.0, pad=20, fontsize=12, fontweight='bold')

plt.savefig("img/experiment_C_hubs.png", dpi=300, bbox_inches='tight')
plt.close()